In [34]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import random

In [35]:
# Заголовки, чтобы сделать запрос более "человеческим"
headers = {
       "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
       "Accept-Language": "en-US,en;q=0.5",
       "Referer": "https://google.ru/",
   }

base_url = "https://www.tripadvisor.ru/Hotels-g293974-Istanbul-Hotels.html"

In [15]:
# response2 = requests.get('https://www.tripadvisor.ru/Hotel_Review-g293974-d294949-Reviews-Pera_Palace_Hotel-Istanbul.html', headers=headers)
# soup2 = BeautifulSoup(response2.text, 'lxml')


In [28]:
# dict_hotels={}

# # Находим элементы, содержащие информацию об отелях
# # hotels_name = soup2.find('div', class_='WMndO f').find('h1', {'id':'HEADING'}).text.strip()
# hotels_name = soup2.find_all('span', class_='biGQs _P pZUbB avBIb osNWb')[-1].text.strip()
    
# dict_hotels['Название отеля en'] = hotels_name
# print(dict_hotels)

# # try:
# #     stars = soup2.find('div', class_='CMiVw _R MC S4 _a H').find('span').text.strip()
# #     dict_hotels['Класс отеля'] = stars
# # except:
# #     dict_hotels['Класс отеля'] = None
# # print(dict_hotels)

# # benefit_info = soup2.find_all('div', class_='uqMDf z RHFuI YGfmd YQkjl')
# # if dict_hotels['Класс отеля'] != None:
# #     ben_i = 1
# # else:
# #     ben_i = 0
# # for benefit in benefit_info:
# #     # print(benefit.text.strip())
# #     for b in benefit.find_all('div', class_='MTDER f v Ci S2 H2 b Wf')[ben_i:]:
# #         print(b.text.strip())
# #         b_data_list = []
# #         for b_data in benefit.find_all('div', class_='CMiVw _R MC S4 _a H')[ben_i:]:
# #             b_data_list.append(b_data.text.strip())
# #         dict_hotels[b.text.strip()] = b_data_list
# #     ben_i=0

# # dict_hotels

{'Название отеля en': 'Pera Palace Hotel'}


In [36]:
def scrape_hotel_info(url):
    response1 = requests.get(url, headers=headers)
    if response1.status_code != 200:
        print(f"Ошибка: {response1.status_code}")
        return
    
    soup1 = BeautifulSoup(response1.text, 'lxml')
    rating = soup1.find_all('span', class_='kJyXc P')
    rating_all = soup1.find('ul', class_='teJcc S4 Cj')
    dict_hotels = {}
    
    adress = soup1.find('div', class_='FhOgt H3 f u KmNnI').text.strip()
    dict_hotels['Адрес'] = adress

    # Находим элементы, содержащие информацию об отелях
    hotels_name = soup1.find_all('span', class_='biGQs _P pZUbB avBIb osNWb')[-1].text.strip()
    
    dict_hotels['Название отеля en'] = hotels_name
    
    try:
        marks = soup1.find('div', class_='dGsKv Xe f P').find('span', class_='kJyXc P').text.strip()
        dict_hotels['Общая оценка'] = marks
    except:
        dict_hotels['Общая оценка'] = None
    
    try:
        reviews = soup1.find('span', class_='biGQs _P pZUbB KxBGd').text.replace('\xa0','').strip()
        dict_hotels['Кол-во отзывов'] = reviews
    except:
        dict_hotels['Кол-во отзывов'] = None

    circles = soup1.find_all('div', class_='ZuFMR f')
    for circle in circles:
        circle_name = circle.find('div', class_='CdRej').text.strip()
        circle_data = circle.find('title').text.strip()
        dict_hotels[circle_name] = circle_data
    
    # Кол-во звезд отеля
    try:
        stars = soup1.find('div', class_='CMiVw _R MC S4 _a H').find('span').text.strip()
        dict_hotels['Класс отеля'] = stars
    except:
        dict_hotels['Класс отеля'] = None

    benefit_info = soup1.find_all('div', class_='uqMDf z RHFuI YGfmd YQkjl')
    if dict_hotels['Класс отеля'] != None:
        ben_i = 1
    else:
        ben_i = 0
    for benefit in benefit_info:
        # print(benefit.text.strip())
        for b in benefit.find_all('div', class_='MTDER f v Ci S2 H2 b Wf')[ben_i:]:
            print(b.text.strip())
            b_data_list = []
            for b_data in benefit.find_all('div', class_='CMiVw _R MC S4 _a H')[ben_i:]:
                b_data_list.append(b_data.text.replace('\xa0',' ').strip())
            dict_hotels[b.text.strip()] = b_data_list
        ben_i=0

    try:    
        services = soup1.find_all('div', class_='Jevoh f K')
        services_list = [[],[],[]]
        for s in range(len(services)):
            for service in services[s].find_all('div', class_='gFttI f ME Ci H3 _c'):
                services_list[s].append(service.text.replace('\xa0', ' ').strip())
        dict_hotels['Услуги и удобства в отеле'] = services_list[0]
        dict_hotels['Оснащенность номера'] = services_list[1]
        dict_hotels['Типы номеров'] = services_list[2]
    except:
        dict_hotels['Услуги и удобства в отеле'] = None
        dict_hotels['Оснащенность номера'] = None
        dict_hotels['Типы номеров'] = None

    print(dict_hotels)
    return dict_hotels    
    # print(rating_all)

In [37]:
# Функция для извлечения информации о отелях
def scrape_hotels(url):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Ошибка: {response.status_code}")
        return
    
    soup = BeautifulSoup(response.text, 'lxml')
    
    # Находим элементы, содержащие информацию об отелях
    hotels = soup.find('ol', class_='tAknw f e').find_all('li')
    # print('hotels from def', hotels)
    
    counter = 0
    hotel_list = []
    for hotel in hotels:
        # if counter>1: break
        counter+=1
        hotel_name = hotel.find('h3').text.strip()
        
        hotel_link = 'https://www.tripadvisor.ru' + hotel.find('a')['href']
        print(f'Отель: {hotel_name}\nСсылка: {hotel_link}\n')
        time.sleep(random.randint(10, 20)) # Задержка между запросами
        dict_hotels = scrape_hotel_info(hotel_link)

        dict_hotels['Название_отеля'] = hotel_name
        hotel_list.append(dict_hotels)
        
    print('counter: ', counter)
    return hotel_list
    

In [5]:
# hotel_list1 = scrape_hotels(base_url)

Отель: 1. Romance Istanbul Hotel
Ссылка: https://www.tripadvisor.ru/Hotel_Review-g293974-d8364987-Reviews-Romance_Istanbul_Hotel-Istanbul.html



KeyboardInterrupt: 

In [55]:
# hotel_list1

NameError: name 'hotel_list1' is not defined

In [38]:

# Переменная для хранения URL страницы
hotel_list = []
current_url = base_url
c = 0
# Основной цикл для скрапинга данных со всех страниц
while current_url:
    c+=1
    hotel_list_i = scrape_hotels(current_url)
    
    # Заходим на следующую страницу
    time.sleep(random.randint(10, 20)) # Задержка между запросами
    response = requests.get(current_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Поиск ссылки на следующую страницу
    next_button = soup.find('a', {'class':'BrOJk u j z _F wSSLS tIqAi unMkR', 'aria-label':'Next page'})
    print(next_button)
    if c==31:
        print('second page')
        break
    if next_button and 'href' in next_button.attrs:
        current_url = 'https://www.tripadvisor.ru' + next_button['href']
        print(current_url)
    else:
        current_url = None
    
    hotel_list.append(hotel_list_i)
    time.sleep(random.randint(3, 15)) # Задержка между запросами 

Отель: 1. Romance Istanbul Hotel
Ссылка: https://www.tripadvisor.ru/Hotel_Review-g293974-d8364987-Reviews-Romance_Istanbul_Hotel-Istanbul.html

СТИЛЬ ОТЕЛЯ
Языки, используемые в общении
{'Адрес': 'Hudavendigar Cd. No:5, Стамбул 34110 Турция', 'Название отеля en': 'Romance Istanbul Hotel', 'Общая оценка': '5,0', 'Кол-во отзывов': '5163 отзыва', 'Расположение': '5,0 из 5 кружков', 'Номера': '4,9 из 5 кружков', 'Цена/качество': '4,9 из 5 кружков', 'Чистота': '4,9 из 5 кружков', 'Обслуживание': '5,0 из 5 кружков', 'Класс отеля': '4,0 из 5 звезд', 'СТИЛЬ ОТЕЛЯ': ['Романтический', 'Очаровательный'], 'Языки, используемые в общении': ['Русский, Английский, Турецкий'], 'Услуги и удобства в отеле': ['Парковка', 'Бесплатный WiFi', 'Фитнес-центр', 'Бассейн', 'Бесплатный завтрак', 'Бесплатное размещение для детей', 'Трансфер из/до аэропорта', 'Бизнес-центр с Wi-Fi', 'Wi-Fi', 'Раздевалки в фитнес- и спа-центрах', 'Сауна', 'Крытый бассейн', 'Бар/лаунж', 'Ресторан', 'Возможен завтрак', 'Завтрак по тип

In [39]:
# Создание DataFrame из списка списков словарей
# Объединяем все внутренние списки в один
flattened_hotels = [item for sublist in hotel_list for item in sublist]

# Создаем DataFrame
df_hotels = pd.DataFrame(flattened_hotels)

In [40]:
df_hotels.to_csv('hotels_info.csv')